In [ ]:
import random
import networkx as nx
import time
import numpy as np
from datetime import datetime

# Define the graph

G = nx.Graph()
G.add_node("London")
G.add_node("Amsterdam")
G.add_node("Berlin")
G.add_node("Cologne")
G.add_node("Frankfurt")
G.add_node("Milan")
G.add_node("Lyon")
G.add_node("Barcelona")
G.add_node("Madrid")
G.add_node("Paris")
G.add_node("Rome")
G.add_node("Brussels")
G.add_edge("London", "Paris", cost = 98, time = 136)
G.add_edge("London", "Brussels", cost=48, time = 105)
G.add_edge("Brussels", "Paris", cost = 80, time = 82)
G.add_edge("Brussels", "Amsterdam", cost = 40, time = 120)
G.add_edge("Paris", "Madrid", cost = 380, time = 225)
G.add_edge("Paris", "Barcelona", cost = 400, time = 390)
G.add_edge("Paris", "Lyon", cost = 185, time = 112)
G.add_edge("Paris", "Frankfurt", cost = 345, time = 480)
G.add_edge("Madrid", "Barcelona", cost = 98, time = 150)
G.add_edge("Barcelona", "Lyon", cost = 320, time = 260)
G.add_edge("Lyon", "Milan", cost = 180, time = 234)
G.add_edge("Milan", "Frankfurt", cost = 240, time = 454)
G.add_edge("Milan", "Rome", cost = 125, time = 168)
G.add_edge("Frankfurt", "Berlin", cost = 125, time = 232)
G.add_edge("Frankfurt", "Cologne", cost = 40, time = 120)
G.add_edge("Amsterdam", "Cologne", cost = 40, time = 120)
G.add_edge("Amsterdam", "Berlin", cost = 235, time = 364)

In [ ]:
G_new = G.copy()

In [ ]:
#Making the graph complete, using djikstra
for i in G.nodes():
  for j in G.nodes():
    #Check every node but only calculate paths if i and j are different nodes
    if i !=j:
      #Initialize cost and time as 0
      costo = 0
      tiempo = 0
      #Only add an edge if it is not already present
      if ((str(i),str(j)) or (str(i),str(j))) not in G_new.edges():
        #For every element in the shortest path except for the last (because we will be adding +1)
        for k in range(0, len(nx.dijkstra_path(G, i, j, weight='time'))-1):
          #If the edge (k,k+1) exists
          try:
            #Add the cost of going from k to k+1
            costo = costo + G.get_edge_data(nx.dijkstra_path(G, i, j, weight='time')[k],nx.dijkstra_path(G, i, j, weight='time')[k+1])['cost']
            #Add the time of going from k to k+1
            tiempo = tiempo + G.get_edge_data(nx.dijkstra_path(G, i, j, weight='time')[k],nx.dijkstra_path(G, i, j, weight='time')[k+1])['time']
          #If the edge (k,k+1) doesn't exist, but rather it is (k+1,k) (inconsequential in an undirected graph, but is needed for networkx syntax)
          except:
            #Add the cost of going from k+1 to k
            costo = costo + G.get_edge_data(nx.dijkstra_path(G, i, j, weight='time')[k+1],nx.dijkstra_path(G, i, j, weight='time')[k])['cost']
            #Add the time of going from k to k+1
            tiempo = tiempo + G.get_edge_data(nx.dijkstra_path(G, i, j, weight='time')[k+1],nx.dijkstra_path(G, i, j, weight='time')[k])['time']
        #Add the new edge with corresponding cost and time
        G_new.add_edge(i,j, cost = costo, time = tiempo)
#Check graph density to make sure it is complete, if density = 1, then the graph is complete
print(nx.density(G_new))

1.0


In [ ]:
#Function to make the first population
def initialize_population(cities, population_size):
    #Initialize first population as empty
    population = []
    #Make as many individuals as wantes
    for i in range(population_size):
        #Randomly make a list of the cities
        individual = cities[np.random.choice(list(range(len(cities))), len(cities), replace=False)]
        #Add the individual to the population
        population.append(individual)
    #Return the population
    return np.array(population)

In [ ]:
#Function to evaluate the fitness of an individual
def evaluate_fitness(cities, cities_graph):
    #Initialize time and cost as 0
    totalcost = 0
    totaltime = 0
    #For every city in the individual
    for i in range(len(cities)-1):
        a = cities[i]
        b = cities[i+1]
        try:
          #Get the cost of going from a to b if the edge (a,b) exists
          totalcost += cities_graph.get_edge_data(a,b)['cost']
          #Get the time of going from a to b if the edge (a,b) exists
          totaltime += cities_graph.get_edge_data(a,b)['time']
        except:
          #Get the cost of going from b to a if the edge (b,a) exists
          totalcost += cities_graph.get_edge_data(b,a)['cost']
          #Get the cost of going from b to a if the edge (b,a) exists
          totaltime += cities_graph.get_edge_data(b,a)['time']
    #Add the cost and time of returning to the original city from the last city
    try:
      totalcost += cities_graph.get_edge_data(cities[0],cities[-1])['cost']
      totaltime += cities_graph.get_edge_data(cities[0],cities[-1])['time']
    except:
      totalcost += cities_graph.get_edge_data(cities[-1],cities[0])['cost']
      totaltime += cities_graph.get_edge_data(cities[-1],cities[0])['time']
    #Heavily penalize solutions that exceed 72 hours
    if totaltime > 4320:
      totalcost += 990000
    #Return the fitness proportional to the cost (higher costs have lower fitness)
    return 1000 - totalcost/1000

In [ ]:
#Function to get the fitness of a population
def get_population_fitness(population, cities_graph):
    #Initialize a fitness list as a list of zeroes
    fitness_list = np.zeros(len(population))

    #Looping over all solutions computing the fitness for each solution
    for i in  range(len(population)):
        fitness_list[i] = evaluate_fitness(population[i], cities_graph)

    return fitness_list

In [ ]:
#Function to select parents from a population that will mate
def parent_selection(population, fitness_list):
    #Get the probability of mating
    total_fit = fitness_list.sum()
    prob_list = fitness_list/total_fit
    
    #Notice there is the chance that a parent mates with oneself
    parent_list_a = np.random.choice(list(range(len(population))), len(population),p=prob_list, replace=True)
    parent_list_b = np.random.choice(list(range(len(population))), len(population),p=prob_list, replace=True)
    
    parent_list_a = population[parent_list_a]
    parent_list_b = population[parent_list_b]
    
    
    return np.array([parent_list_a,parent_list_b])

In [ ]:
#Function to mate the parents
def mate_parents(parent_a, parent_b):
    #Get a random number of cities from parent a (up to 6, half the cities)
    offspring = parent_a[0:random.randint(1,6)]
    #Start adding the missing cities in the order parent b has them
    for city in parent_b:
        #Only add the cities that are missing
        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring

In [ ]:
#Function to mate a population
def mate_population(potential_parents_list):
    #Initialize the new population as empty
    new_population = []
    #Create offspring
    for i in range(potential_parents_list.shape[1]):
        parent_a, parent_b = potential_parents_list[0][i], potential_parents_list[1][i]
        offspring = mate_parents(parent_a, parent_b)
        new_population.append(offspring)
        
    return new_population

In [ ]:
#Function to mutate an individual
def mutate_offspring(offspring, mutation_rate):
    for q in range(int(len(offspring)*mutation_rate)):
        a = np.random.randint(0,len(offspring))
        b = np.random.randint(0,len(offspring))
        #Swap the order of two cities
        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring

In [ ]:
#Function to mutate a population
def mutate_population(new_population_set, mutation_rate):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring, mutation_rate))
    return np.array(mutated_pop)

In [ ]:
#Function to get the total time of a solution
def get_solution_time(solution, solution_graph):
  totaltime = 0
  for i in range(len(solution)-1):
      a = solution[i]
      b = solution[i+1]
      try:
        totaltime += solution_graph.get_edge_data(a,b)['time']
      except:
        totaltime += solution_graph.get_edge_data(b,a)['time']
  try:
    totaltime += solution_graph.get_edge_data(solution[0],solution[-1])['time']
  except:
    totaltime += solution_graph.get_edge_data(solution[-1],solution[0])['time']
  return totaltime

In [ ]:
population_size = 500
mutation_rate = 0.2
cities = np.array(list(G_new.nodes()))

In [ ]:
#Creating an array to store the best solution
best_solution = [-1,np.inf,np.array([])]
#Set number of iterations
num_of_iter = 2000
#Create the initial population
initial_population = initialize_population(cities, population_size)
#Go through the process of creating a new one
fitnesslist = get_population_fitness(initial_population,G_new)
parent_list = parent_selection(initial_population,fitnesslist)
new_population = mate_population(parent_list)
#Get the second population as the input for the new one
mutated_population = mutate_population(new_population, mutation_rate)
for i in range(num_of_iter):
    #Print the progress
    if i%100==0: print("Porcentaje completado =", int(i/num_of_iter * 100), "| fitness de la mejor solución = ", fitnesslist.max(),
                       "| fitness promedio de la población evaluada =", fitnesslist.mean(), "| encontrada:", datetime.now().strftime("%d/%m/%y %H:%M"))
    #Get the fitness of the input population
    fitnesslist = get_population_fitness(mutated_population,G_new)
    
    #Saving the best solution
    if fitnesslist.min() < best_solution[1]:
        #Save on which iteration it was obtained
        best_solution[0] = i
        #Save the cost of the solution
        best_solution[1] = (1000 - fitnesslist.max())*1000
        #Save the solution
        best_solution[2] = np.array(mutated_population)[fitnesslist.max() == fitnesslist]
    #Create new parents from previous population
    parent_list = parent_selection(mutated_population, fitnesslist)
    #Create new population
    new_population = mate_population(parent_list)
    #Mutate the new population
    mutated_population = mutate_population(new_population, mutation_rate)

Porcentaje completado = 0 | fitness de la mejor solución =  997.476 | fitness promedio de la población evaluada = 69.015414 | encontrada: 01/05/23 21:31
Porcentaje completado = 5 | fitness de la mejor solución =  997.581 | fitness promedio de la población evaluada = 233.736876 | encontrada: 01/05/23 21:31
Porcentaje completado = 10 | fitness de la mejor solución =  997.566 | fitness promedio de la población evaluada = 249.55779199999998 | encontrada: 01/05/23 21:31
Porcentaje completado = 15 | fitness de la mejor solución =  997.461 | fitness promedio de la población evaluada = 255.559166 | encontrada: 01/05/23 21:31
Porcentaje completado = 20 | fitness de la mejor solución =  997.501 | fitness promedio de la población evaluada = 265.47303000000005 | encontrada: 01/05/23 21:31
Porcentaje completado = 25 | fitness de la mejor solución =  997.471 | fitness promedio de la población evaluada = 265.433908 | encontrada: 01/05/23 21:32
Porcentaje completado = 30 | fitness de la mejor solución

In [ ]:
#Get the solution with the intermediary cities
solution_with_intermediaries = []
#For each city in the solution
for i in range(0, len(best_solution[2][0])-1):
  #Append the city to the list
  solution_with_intermediaries.append(best_solution[2][0][i])
  #Get the shortes path in the original graph from city i to city i+1
  pathlist = list(nx.dijkstra_path(G, best_solution[2][0][i], best_solution[2][0][i+1], weight='time'))
  #Remove from the path cities i and i+1
  pathlist = pathlist[1:-1]
  #For each city in the path
  for j in pathlist:
    #Append the city
    solution_with_intermediaries.append(j)
#For the last city, get the path to return to the origin
pathlist = list(nx.dijkstra_path(G, best_solution[2][0][-1], best_solution[2][0][0], weight='time'))
#Remove the origin city (last in the shortest path)
pathlist = pathlist[:-1]
for j in pathlist:
  #Append the cities
  solution_with_intermediaries.append(j)

In [ ]:
print("Best solution without including intermediary cities: ")
for i in best_solution[2][0]:
  print(i + ' to ', end = '')
#Print the origin city
print(best_solution[2][0][0])
print('')
print("Best solution including intermediary cities: ")
for i in solution_with_intermediaries:
  print(i + ' to ', end = '')
#Print the origin city
print(best_solution[2][0][0])
print('')
print("Cost of travel: ")
print(int(best_solution[1]), 'euros')
print('')
print("Time of travel: ")
print(get_solution_time(best_solution[2][0], G_new), 'minutes, or',get_solution_time(best_solution[2][0], G_new)/60, 'hours')

Best solution without including intermediary cities: 
Berlin to Amsterdam to Cologne to Paris to Madrid to Barcelona to Rome to Milan to Lyon to Brussels to London to Frankfurt to Berlin

Best solution including intermediary cities: 
Berlin to Amsterdam to Cologne to Amsterdam to Brussels to Paris to Madrid to Barcelona to Lyon to Milan to Rome to Milan to Lyon to Paris to Brussels to London to Brussels to Amsterdam to Cologne to Frankfurt to Berlin

Cost of travel: 
2448 euros

Time of travel: 
3241 minutes, or 54.016666666666666 hours
